In [47]:
import numpy as np
import cv2
import torch
import torch.utils.data
import os
import pickle
import json

In [49]:
tsfm_path = '/home/xymeng/Repo/jax3d/jax3d/projects/mobilenerf/get_rays/rays_inverse_false.json'
with open(tsfm_path, 'r') as jf:
    data_inv_f = json.load(jf)

tsfm_path_t = '/home/xymeng/Repo/jax3d/jax3d/projects/mobilenerf/get_rays/rays_inverse_true.json'
with open(tsfm_path_t, 'r') as jft:
    data_inv_t = json.load(jft)

FileNotFoundError: [Errno 2] No such file or directory: '/home/xymeng/Repo/jax3d/jax3d/projects/mobilenerf/get_rays/rays_inverse_false.json'

In [3]:
data_inv_f = torch.tensor(data_inv_f)

In [4]:
data_inv_f.shape

torch.Size([512, 512, 3, 1])

In [5]:
data_inv_t = torch.tensor(data_inv_t)
data_inv_t.shape

torch.Size([512, 512, 3, 1])

In [6]:
torch.unique(data_inv_f[...,0,:]==data_inv_t[...,0,:])

tensor([True])

In [12]:
torch.unique(data_inv_f[...,1:,:]+data_inv_t[...,1:,:])

tensor([0., 1.])

In [14]:
data_inv_f[...,1:,:].shape, data_inv_t[...,1:,:].shape

(torch.Size([512, 512, 2, 1]), torch.Size([512, 512, 2, 1]))

In [24]:
abs(data_inv_f[...,1:,:]).mean(), # data_inv_t[...,1:,:]
# (data_inv_f[...,2:3,:]+data_inv_t[...,2:3,:]).sum()

(tensor(128.5000),)

In [82]:
focal = 527
width=height=512

k_zju = np.array([[537.14068604,   0.        , 271.41711426],
       [  0.        , 537.71148682, 242.44180298],
       [  0.        ,   0.        ,   1.        ]])

k = np.array([
      [1./focal, 0, -.5 * width / focal],
      [0, -1./focal, .5 * height / focal],
      [0, 0, -1.],
  ])
kneg = np.array([
      [1./focal, 0, .5 * width / focal],
      [0, 1./focal, -.5 * height / focal],
      [0, 0, 1.],
  ])

neg = np.array([
      [1., 0, 0],
      [0, -1., 0],
      [0, 0, -1.],
  ])

np.linalg.inv(k_zju), k, kneg, np.matmul(kneg, neg), np.matmul(np.linalg.inv(k_zju), neg), np.matmul(k_zju, np.linalg.inv(k_zju))

(array([[ 0.00186171,  0.        , -0.50529986],
        [ 0.        ,  0.00185973, -0.45087711],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ 0.00189753,  0.        , -0.4857685 ],
        [ 0.        , -0.00189753,  0.4857685 ],
        [ 0.        ,  0.        , -1.        ]]),
 array([[ 0.00189753,  0.        ,  0.4857685 ],
        [ 0.        ,  0.00189753, -0.4857685 ],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ 0.00189753,  0.        , -0.4857685 ],
        [ 0.        , -0.00189753,  0.4857685 ],
        [ 0.        ,  0.        , -1.        ]]),
 array([[ 0.00186171,  0.        ,  0.50529986],
        [ 0.        , -0.00185973,  0.45087711],
        [ 0.        ,  0.        , -1.        ]]),
 array([[1.00000000e+00, 0.00000000e+00, 5.68434189e-14],
        [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]))

In [38]:
## actual inv_K for ZJU
DeviceArray([[ 0.00186171,  0.        ,  0.50529987],
             [ 0.        , -0.00185973,  0.4508771 ],
             [ 0.        ,  0.        , -1.        ]], dtype=float32)


In [61]:
inverse_y_mul = np.array([
      [1., 0, 0],
      [0, -1., 0],
      [0, 0, -1.],
    ])
K_zju = np.asarray([[537.1407,   0.0000, 271.4171],
            [  0.0000, 537.7115, 242.4418],
            [  0.0000,   0.0000,   1.0000]])
inv_K = np.linalg.inv(K_zju)
# st()
np.matmul(inv_K, inverse_y_mul)

array([[ 0.00186171,  0.        ,  0.50529982],
       [ 0.        , -0.00185973,  0.4508771 ],
       [ 0.        ,  0.        , -1.        ]])

In [43]:
torch.unique(kt==kf)

tensor([True])

In [74]:
xyz = torch.rand(10,10,3,1)
xyz1 = xyz.clone()
xyz1[...,1:,:] = xyz1[...,1:,:]*(-1)
xyz2 = torch.matmul(torch.tensor(neg).float(), xyz.float())
torch.tensor(neg), torch.unique(xyz2==xyz1)

(tensor([[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]], dtype=torch.float64),
 tensor([True]))

In [80]:
xyz3 = torch.matmul(torch.matmul(torch.tensor(k_zju).float(), torch.tensor(neg).float()), xyz.float())
xyz4 = torch.matmul(torch.tensor(k_zju).float(), xyz2)
torch.unique(xyz3==xyz4)

tensor([True])